# Setup Database Connection

In [1]:
import pandas as pd
from pandas import DataFrame as df

In [2]:
from os import environ

try:
    neo4juser = environ['NEO4JBMUSR']
except KeyError:
    print("Using default neo4j username.")
    neo4juser = 'neo4j'
try:
    neo4jpswd = environ['NEO4JBMPSWD']
except KeyError:
    print("Using default neo4j password.")
    neo4jpswd =  'neo4j'


Using default neo4j username.


In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph(host="localhost", user=neo4juser, password=neo4jpswd)

tx = graph.begin()

## Test the connection

In [4]:
graph.dbms

<DBMS uri='http://localhost:7474/'>

In [5]:
tx = graph.begin()

In [6]:
a = Node("Person", name="Alice")
tx.create(a)
tx.commit()

In [7]:
df(graph.data('MATCH (n1) RETURN n1.name, labels(n1)'))

,labels(n1),n1.name
0,[Person],Alice


In [8]:
graph.exists(a)

True

In [9]:
a.has_label('Person')

True

## Create Policies

In [10]:
def createPolicy(name):
    return Node("Policy", name=name)

def createArg(policy):
    arg = Node("Argument", name=policy['name'])
    return (arg, Relationship(arg, 'instance', policy))

def commitAll(args):
    tx = graph.begin()
    for arg in args:
        tx.create(arg)
    tx.commit()

In [11]:
p = createPolicy('Ban all guns')
(arg, rel) = createArg(p)

In [12]:
commitAll([p, arg, rel])

In [13]:
def numVal(string):
    try:
        x = int(string.strip())
    except:
        raise ValueError('Please enter a number from 1-10, not \'{}\'.'.format(string))
    if x<11 and x >0:
        return x
    else:
        raise ValueError('Please enter a number from 1-10, not \'{}\'.'.format(x))
    

def inputVal(string):
#   Make sure no fucked up strings stuff
    return string

# flatten list of lists to a single list
def flatten(ll):
    return [x for l in ll for x in l]

In [15]:
# numVal('     80')

In [16]:
def setConRel(sub_args, arg, pro_con):
    confidence_string = 'How confident from 1-10 are you in \'{}\'? '
    rel_string = 'From 1-10 how well does the statement \'{}\' confirm \'{}\'? '
    new_sub_args = []
    new_sub_rels = []
    for sub_arg in sub_args:
        valid_num = False
        
        while (not valid_num):
            try:
                confidence = numVal(input(confidence_string.format(sub_arg['name'])) )
                valid_num = True
            except ValueError:
                print('Please enter a number from 1-10.')
        
        sub_arg['confidence'] = confidence
        
        valid_num = False
        
        while (not valid_num):
            try:
                rel = numVal(input(rel_string.format(sub_arg['name'], arg['name'])) )
                valid_num = True
            except ValueError:
                print('Please enter a number from 1-10.')
        
        sub_rel = Relationship(arg, pro_con, sub_arg, relevance=rel)
        
        new_sub_args.append(sub_arg)
        new_sub_rels.append(sub_rel)
        
    return (new_sub_args, new_sub_rels)
        
        
def getArgs(arg):
    sub_args = []
    sub_rels = []
    
#   Get all sub args
    input_string = 'Enter {} statement names for \'{}\' (type enter when done typing subarguments): '

#   Pro Args
    name = inputVal(input(input_string.format('supporting', arg['name'])) )
    while(len(name) > 0):
        sub_arg = Node("Statement", name=name)
        sub_args.append(sub_arg)
        
        name = inputVal(input(input_string.format('supporting', arg['name'])) )
    
#   Set sub arguments to adjusted args
    (sub_args, sub_rels) = setConRel(sub_args, arg, 'Pro')
    
    con_sub_args = []
    con_sub_rels = []
#   Con args
    name = inputVal(input(input_string.format('counter', arg['name'])) )
    while(len(name) > 0):
        con_sub_arg = Node("Statement", name=name)
        con_sub_args.append(con_sub_arg)
        
        name = inputVal(input(input_string.format('counter', arg['name'])) )
        
    #   Set sub arguments to adjusted args
    (con_sub_args, con_sub_rels) = setConRel(con_sub_args, arg, 'Con')

#   add pro and con sides of the argument together
    sub_args += con_sub_args
    sub_rels += con_sub_rels
    
    new_sub_args = []
    new_sub_rels = []
    if len(sub_args) > 0:
        for sub_arg in sub_args:
            (temp_args, temp_rels) = getArgs(sub_arg)
            new_sub_args += temp_args
            new_sub_rels += temp_rels
    
    return (sub_args+new_sub_args, sub_rels+new_sub_rels)

In [17]:
rels_args = flatten(getArgs(arg))

Enter supporting statement names for 'Ban all guns' (type enter when done typing subarguments): Lower gun crimes
Enter supporting statement names for 'Ban all guns' (type enter when done typing subarguments): 
How confident from 1-10 are you in 'Lower gun crimes'? 8
From 1-10 how well does the statement 'Lower gun crimes' confirm 'Ban all guns'? 10
Enter counter statement names for 'Ban all guns' (type enter when done typing subarguments): Check Government
Enter counter statement names for 'Ban all guns' (type enter when done typing subarguments): Self Defense
Enter counter statement names for 'Ban all guns' (type enter when done typing subarguments): Gun Industry/ Culture
Enter counter statement names for 'Ban all guns' (type enter when done typing subarguments): 
How confident from 1-10 are you in 'Check Government'? 7
From 1-10 how well does the statement 'Check Government' confirm 'Ban all guns'? 9
How confident from 1-10 are you in 'Self Defense'? 3
From 1-10 how well does the sta

In [18]:
commitAll(rels_args)